In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
train = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-training.csv")
test = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-test.csv")
# print(train.columns)
# print(test.columns)

In [ ]:
# 合并处理
df = pd.concat([train, test], ignore_index=True)
df.head()

In [ ]:
# 检查缺失值情况
df.isnull().sum()

In [ ]:
raw_columns = df.columns
for i, col in enumerate(raw_columns):
    print("这是第 %2d 个变量: %s" % (i, col))
    
# 第0个变量是无效的用来作为唯一标识符的值

# 01.目标变量：SeriousDlqin2yrs

备注：逾期90天及以上者

In [ ]:
sns.countplot(df.SeriousDlqin2yrs)
df['y'] = df.SeriousDlqin2yrs

# 02. RevolvingUtilizationOfUnsecuredLines

备注：除分期贷之外信用卡剩余额度占比

In [ ]:
df.RevolvingUtilizationOfUnsecuredLines.describe()  # 按理不该存在大于1的值

In [ ]:
df['RUOUL'] = df.RevolvingUtilizationOfUnsecuredLines.copy()
# df.loc[df.RUOUL > 1, 'RUOUL'] = 1
df.RUOUL.describe()

# 分布
plt.figure(figsize=(16, 9))
sns.distplot(df.RUOUL, kde=False)

In [ ]:
# 分箱操作
N = 11
df['RUOUL'] = pd.qcut(df.RUOUL, N, labels=[i/10 for i in range(N)], duplicates='drop')  # 'drop' vs. 'raise'
print(df.RUOUL.value_counts())

In [ ]:

# plot
sns.countplot(df['RUOUL'], hue=df['SeriousDlqin2yrs'])

# 03. age

备注：年龄

In [ ]:
df['AGE'] = df.age.copy()

# plot
sns.countplot(df.AGE)

In [ ]:
# 分箱操作
bins = [0, 30, 40, 50, 60, 70, 120]
# print(bins)
labels = [i/(len(bins)-2) for i in range(len(bins) - 1)]
# print(labels)
df.AGE = pd.cut(df.AGE, bins, right=0, labels=labels)
# plot
sns.countplot(x='AGE', hue='SeriousDlqin2yrs', data=df)

# 04.NumberOfTime30-59DaysPastDueNotWorse

备注：有逾期30-59天的记录次数

In [ ]:
df['NOT30D'] = df['NumberOfTime30-59DaysPastDueNotWorse'].copy()

# plot
sns.countplot(df.NOT30D)

In [ ]:
# 直接进行二类映射表示有此逾期记录
df.loc[df.NOT30D > 0, 'NOT30D'] = 1
sns.countplot(df.NOT30D)

# 05.DebtRatio


备注：月支出/月收入占比

In [ ]:
df['DR'] = df.DebtRatio.copy()
df['DR'].describe()  # 存在异常值

In [ ]:
plt.figure(figsize=(16, 9))
sns.boxplot(y=df['DR'])

In [ ]:
# 分箱操作
N = 11
df['DR'] = pd.qcut(df.DR, N, labels=[i/10 for i in range(N)], duplicates='drop')  # 'drop' vs. 'raise'

In [ ]:
# plot
sns.countplot(df.DR, hue=df.SeriousDlqin2yrs)
# sns.boxplot(y=df['DR'])
df.DR.value_counts()

# 06.MonthlyIncome

备注：月收入，有大量的缺失值，先用众数进行填充

In [ ]:
df['MI'] = df.MonthlyIncome.copy()
df['MI'] = df.MI.fillna(df.MI.median())
df.MI.describe()

In [ ]:
# 分箱操作
N = 11
df['MI'] = pd.qcut(df.MI, N, labels=[i/10 for i in range(N-1)], duplicates='drop')  # 'drop' vs. 'raise'

In [ ]:
# plot
sns.countplot(df.MI)  # , hue=df.SeriousDlqin2yrs
df.MI.value_counts()

# 07.NumberOfOpenCreditLinesAndLoans

未到期贷款额：属于贷款但还没到偿还的时间

In [ ]:
df['NOOCLAL'] = df.NumberOfOpenCreditLinesAndLoans.copy()
df.NOOCLAL.describe()

In [ ]:
# 分箱操作
N = 11
df['NOOCLAL'] = pd.qcut(df.NOOCLAL, N, labels=[i/10 for i in range(N)], duplicates='drop')  # 'drop' vs. 'raise'

In [ ]:
# plot
sns.countplot(df.NOOCLAL)  # , hue=df.SeriousDlqin2yrs
df.NOOCLAL.value_counts()
df.NOOCLAL.isnull().sum()

# 08.NumberOfTimes90DaysLate

备注：有逾期90天的记录次数

In [ ]:
df['NOT90D'] = df.NumberOfTimes90DaysLate.copy()
# 直接进行二类映射表示有此逾期记录
df.loc[df.NOT90D > 0, 'NOT90D'] = 1
sns.countplot(df.NOT90D)

# 09.NumberRealEstateLoansOrLines

备注：抵押贷款和房地产贷款的数量，包括房屋净值信贷额度

In [ ]:
df['NRELO'] = df.NumberRealEstateLoansOrLines.copy()
df['NRELO'].describe()

In [ ]:
# 分组统计
bins = [0, 1, 3, df.NRELO.max()]
labels = [0, 0.5, 1.0]
df['NRELO'] = pd.cut(df['NRELO'], bins, right=0, labels=labels)

# plot
sns.countplot(df.NRELO, hue=df.y)

# 10.NumberOfTime60-89DaysPastDueNotWorse

备注：有逾期60-89天的记录次数

In [ ]:
df['NOT60D'] = df['NumberOfTime60-89DaysPastDueNotWorse'].copy()
# 直接进行二类映射表示有此逾期记录
df.loc[df.NOT60D > 0, 'NOT60D'] = 1
sns.countplot(df.NOT60D)

# 11.NumberOfDependents

备注：除自己之外的家庭受养人数，有缺失值需要填充

In [ ]:
df['NOD'] = df['NumberOfDependents'].copy()
df['NOD'] = df.NOD.fillna(0)
df.NOD.describe()

In [ ]:
# 分组统计
bins = [0, 1, 2, 4, 8, df.NOD.max()]
labels = [0, 0.25, 0.5, 0.75, 1.0]
df['NOD'] = pd.cut(df['NOD'], bins, right=0, labels=labels)

# plot
sns.countplot(df.NOD, hue=df.y)

# 删除不再需要的raw_columns

In [ ]:
df.drop(raw_columns, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.dtypes

# 分离数据集

In [ ]:
df[:len(train)]['y'].value_counts()  # 不均衡数据集

In [ ]:
# 可以通过复制进行数据集升采样
train_0_data = df[:len(train)][df.y == 0]
train_1_data = df[:len(train)][df.y == 1]
print(train_0_data.shape, train_1_data.shape)

In [ ]:
# UpSample
# train_1_data_up = train_0_data.copy()
# print(train_1_data_up.shape)
# for i in np.arange(len(train_0_data)):
#     idx = np.random.randint(0, train_1_data.shape[0])
#     train_1_data_up.iloc[i] = train_1_data.iloc[idx]
train_1_data_up = pd.concat([train_1_data] * 14, ignore_index=True)
print(train_1_data_up.shape)
print(train_1_data_up.iloc[0, :], '\n' + '-'*50)
print(train_1_data_up.iloc[0+train_1_data.shape[0], :])

In [ ]:
print(train_0_data.shape, train_1_data_up.shape)

In [ ]:
train_data = pd.concat([train_0_data, train_1_data_up], ignore_index=True)
print(train_data.isnull().sum())  # 再次检查缺失值
train_data.dropna(axis=0, how='any', inplace=True) # 直接删除
print(train_data[train_data.NRELO.isnull()])

X, y = train_data.iloc[:, 1:].values, train_data.y.values.ravel()
X, y = X.astype('f4'), y.astype('f4')
print(X.dtype, y.dtype)
print(X.shape, y.shape)
print(X[-1], y[-1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=2019)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

# 基准模型

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model = LogisticRegression(C=3, solver='lbfgs').fit(X_train, y_train)
print(model.score(X_val, y_val))
print(roc_auc_score(y_val, model.predict(X_val)))

# XGB

In [ ]:
import xgboost as xgb

# model = xgb.XGBClassifier(
#     max_depth=5,
#     eta=0.025,
#     silent=1,
#     objective='binary:logistic',
#     eval_matric='auc',
#     minchildweight=10.0,
#     maxdeltastep=1.8,
#     colsample_bytree=0.4,
#     subsample=0.8,
#     gamma=0.65,
#     numboostround=391
# )

model = xgb.XGBClassifier(
    max_depth=6,
    eta=1,
    silent=1,
    objective='binary:logistic',
    eval_matric='f1'
)

model.fit(X_train, y_train)
print(model.score(X_val, y_val))
print(roc_auc_score(y_val, model.predict(X_val)))

# 提交结果

In [ ]:
X_test = df[len(train):]
print(X_test.isnull().sum(), '\n' + '-'*100)
print(X_test[X_test.NOD.isnull()])
X_test['NOD'] = X_test['NOD'].astype("float64")  # 为什么会有一个缺失值的出现
X_test['NOD'] = X_test.NOD.fillna(X_test.NOD.median())
X_test = X_test.iloc[:, 1:]
X_test.head()

In [ ]:
y_test = model.predict(X_test.values)

In [ ]:
submission = pd.DataFrame()
submission['ID'] = np.arange(1, len(y_test) + 1)
submission['Probability'] = y_test
submission.to_csv('submission.csv', header=True, index=False)